In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pls

In [2]:
labeled = pd.read_excel('manual_labels.xlsx')
unlabeled = pd.read_excel('manual_unlabeled.xlsx')

In [3]:
labeled.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [4]:
unlabeled['label'] = -1

In [5]:
unlabeled.head()

,Unnamed: 0,Rating,Review,lang,label
1764,1764,5,I just love this game I can't delete it like b...,en,-1
24448,24448,5,I think in the tutorial u should make to where...,en,-1
50689,50689,4,"Needs Better Matchmaking Im in Iron League, a...",en,-1
54637,54637,5,Is a great game,en,-1
20221,20221,5,R3ALLY COOL EXPIRIANCE!! have u ever played th...,en,-1


In [10]:
val_en = pd.read_excel('validation_en.xlsx')

In [14]:
total = pd.concat([labeled[['Review', 'label']], unlabeled[['Review','label']]], axis = 0)

In [15]:
total.label.value_counts()

-1    12800
 4      624
 2      203
 5       74
 0       60
 3       31
 1        8
Name: label, dtype: int64

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3),
                       analyzer = 'word',
                       stop_words = 'english',
                       max_features = 1000,
                       min_df = 2, max_df = 0.9).fit(total.Review)
feats = vect.transform(total.Review).toarray()
labels = total.label.as_matrix()

In [21]:
feats.shape

(13800, 1000)

In [48]:
from sklearn.semi_supervised import label_propagation
from scipy.sparse import csgraph
lp_model = label_propagation.LabelSpreading(kernel = 'knn',gamma = 0.1, n_neighbors = 10,alpha = 0.2, max_iter=15)
lp_model.fit(feats, labels)

/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


LabelSpreading(alpha=0.2, gamma=0.1, kernel='knn', max_iter=15, n_jobs=1,
        n_neighbors=10, tol=0.001)

In [49]:
lp_model.n_iter_

8

In [50]:
pd.Series(lp_model.transduction_).value_counts()

4    9804
0    2567
2     912
5     388
3      88
1      41
dtype: int64

In [51]:
val_en = pd.read_excel('validation_en.xlsx')

In [52]:
m = lp_model

In [53]:
def eval_classifier(input_text,model = m):
    feats = vect.transform([input_text])
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return class_names[prediction[0]]

In [54]:
val_en.Syncronization.apply(eval_classifier)

0    Positive
1     unknown
2         Bug
3    Positive
4     unknown
5     unknown
6    Positive
7     unknown
8     unknown
9    Positive
Name: Syncronization, dtype: object

In [11]:
val_en.Balance.apply(eval_classifier)

NameError: name 'eval_classifier' is not defined

In [55]:
labels_p = lp_model.transduction_

In [79]:
total['label_p'] = labels_p

In [84]:
total.iloc[1000:,:].head(50)

,Review,label,label_p
1764,I just love this game I can't delete it like b...,-1,0
24448,I think in the tutorial u should make to where...,-1,4
50689,"Needs Better Matchmaking Im in Iron League, a...",-1,4
54637,Is a great game,-1,0
20221,R3ALLY COOL EXPIRIANCE!! have u ever played th...,-1,4
28111,Love it 😋😋,-1,4
32633,Loved it,-1,4
37407,Port it over to nintendo switch please!!!!!!!!...,-1,4
16778,IT IS AWESOME! !!!!!!!!!!,-1,4
40535,hello I would like you to add a game mode that...,-1,4


In [57]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [66]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [67]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [68]:
m = OneVsRestClassifier(DummyClassifier()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       0.93      0.85      0.89     10244
           Crash       0.02      0.02      0.02        46
Balance problems       0.00      0.00      0.00         8
 Synchronization       0.02      0.02      0.02       157
        Positive       0.00      0.00      0.00        24
             Bug       0.04      0.05      0.04       496

     avg / total       0.86      0.79      0.83     11040

test scores

                  precision    recall  f1-score   support

         unknown       0.93      0.85      0.89      2556
           Crash       0.00      0.00      0.00        14
Balance problems       0.00      0.00      0.00         0
 Synchronization       0.03      0.02      0.02        46
        Positive       0.00      0.00      0.00         7
             Bug       0.04      0.05      0.05       128

     avg / total       0.86      0.79      0.83      2760



/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 6
  .format(len(labels), len(target_names))
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [77]:
m = MultinomialNB().fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       0.93      0.96      0.95     10244
           Crash       0.12      0.04      0.06        46
Balance problems       0.00      0.00      0.00         8
 Synchronization       0.20      0.15      0.17       157
        Positive       0.22      0.25      0.24        24
             Bug       0.17      0.08      0.11       496

     avg / total       0.88      0.90      0.89     11040

test scores

                  precision    recall  f1-score   support

         unknown       0.93      0.96      0.94      2556
           Crash       0.00      0.00      0.00        14
Balance problems       0.08      0.04      0.06        46
 Synchronization       0.00      0.00      0.00         7
        Positive       0.15      0.06      0.09       128
             Bug       0.00      0.00      0.00         9

     avg / total       0.87      0.89      0.88      2760



/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 7, does not match size of target_names, 6
  .format(len(labels), len(target_names))
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
